# Backtesting a Moving Average Crossover Strategy on SPY, QQQ and NVDA (2000–2025)

This notebook implements and evaluates a simple **moving average crossover** strategy on:

- **SPY** (S&P 500 ETF)  
- **QQQ** (Nasdaq 100 ETF)  
- **NVDA** (NVIDIA Corporation)

We use daily data from **2000-01-01 to 2025-01-01** and test a long-only strategy:

- Go long when the **fast moving average** crosses above the **slow moving average**.  
- Exit (go to cash) when the fast moving average falls below the slow moving average.

For each asset we:

- Build trading signals.  
- Backtest daily returns.  
- Compute performance metrics (CAGR, volatility, Sharpe ratio, max drawdown, hit ratio).  
- Plot the equity curves.

This project demonstrates:

- A Python-based backtesting workflow.  
- Handling market data with `yfinance` and `pandas`.  
- Translating a trading idea into reproducible code and clear visuals.

In [67]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

In [68]:
tickers = ["SPY", "QQQ", "NVDA"]

start_date = "2000-01-01"
end_date = "2025-01-01"

fast_window = 50
slow_window = 200

# 2% approximate annual risk-free rate
risk_free_rate = 0.02

In [69]:
def download_price_data(tickers, start, end):
    """
    Download daily price data from Yahoo Finance and return
    a DataFrame where each column is a ticker.
    """
    all_series = []

    for t in tickers:
        data = yf.download(t, start=start, end=end)

        # Prefer 'Adj Close'; if not available, fall back to 'Close'
        if "Adj Close" in data.columns:
            s = data["Adj Close"]
        elif "Close" in data.columns:
            s = data["Close"]
        else:
            raise ValueError(f"For ticker {t} neither 'Adj Close' nor 'Close' were found.")

        # If for some reason we get a DataFrame, convert to a Series
        if isinstance(s, pd.DataFrame):
            # use the first column
            s = s.iloc[:, 0]

        # Set the final column name in the prices DataFrame
        s.name = t

        all_series.append(s)

    prices = pd.concat(all_series, axis=1)
    return prices

In [70]:
prices = download_price_data(tickers, start_date, end_date)
prices.tail()

C:\Users\Denisse Pareja\AppData\Local\Temp\ipykernel_6580\4143975028.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
C:\Users\Denisse Pareja\AppData\Local\Temp\ipykernel_6580\4143975028.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
C:\Users\Denisse Pareja\AppData\Local\Temp\ipykernel_6580\4143975028.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end)
[*********************100%***********************]  1 of 1 completed


,SPY,QQQ,NVDA
Date,,,
2024-12-24,596.076965,527.965149,140.189468
2024-12-26,596.116638,527.606506,139.899521
2024-12-27,589.841614,520.592957,136.980164
2024-12-30,583.110596,513.669189,137.460052
2024-12-31,580.989136,509.305664,134.260757


In [71]:
def build_ma_crossover_signals(price_series, fast=50, slow=200):
    """
    Build a moving-average crossover strategy:

    Columns:
    - price
    - fast_ma
    - slow_ma
    - position (1 = long, 0 = flat)
    - asset_return
    - strategy_return
    - equity_curve
    """
    df = pd.DataFrame({"price": price_series})

    df["fast_ma"] = df["price"].rolling(window=fast).mean()
    df["slow_ma"] = df["price"].rolling(window=slow).mean()

    # Position: long when the fast MA is above the slow MA
    df["position"] = 0
    df.loc[df["fast_ma"] > df["slow_ma"], "position"] = 1

    df["asset_return"] = df["price"].pct_change().fillna(0.0)

    # Use previous day's position to avoid look-ahead bias
    df["strategy_return"] = df["asset_return"] * df["position"].shift(1).fillna(0.0)

    df["equity_curve"] = (1 + df["strategy_return"]).cumprod()

    return df.dropna()

In [72]:
def backtest_strategy(df, rf_rate=0.0):
    """
    Given a DataFrame with 'strategy_return' and 'equity_curve',
    compute key performance metrics:

    - total_return
    - CAGR
    - annualized volatility
    - Sharpe ratio
    - maximum drawdown
    - hit ratio
    """
    df = df.copy()

    n_days = len(df)
    annual_factor = 252

    # Total return (over 1$ initial capital)
    total_return = df["equity_curve"].iloc[-1] - 1.0

    # CAGR
    cagr = df["equity_curve"].iloc[-1] ** (annual_factor / n_days) - 1

    # Annualized volatility
    vol_annual = df["strategy_return"].std() * np.sqrt(annual_factor)

    # Sharpe ratio (using an annual risk-free rate)
    rf_daily = rf_rate / annual_factor
    excess_daily = df["strategy_return"] - rf_daily
    if excess_daily.std() > 0:
        sharpe = (excess_daily.mean() / excess_daily.std()) * np.sqrt(annual_factor)
    else:
        sharpe = np.nan

    # Max drawdown
    running_max = df["equity_curve"].cummax()
    drawdown = df["equity_curve"] / running_max - 1.0
    max_drawdown = drawdown.min()

    # Hit ratio (percentage of days with positive returns)
    hit_ratio = (df["strategy_return"] > 0).mean()

    metrics = pd.DataFrame(
        {
            "total_return": [total_return],
            "CAGR": [cagr],
            "vol_annual": [vol_annual],
            "Sharpe": [sharpe],
            "max_drawdown": [max_drawdown],
            "hit_ratio": [hit_ratio],
        }
    )

    return df, metrics

In [73]:
# Store strategy results and performance metrics
results = {}
metrics_list = []

for ticker in tickers:
    price_series = prices[ticker].dropna()

    signals = build_ma_crossover_signals(
        price_series,
        fast=fast_window,
        slow=slow_window,
    )

    df_p, metrics = backtest_strategy(signals, rf_rate=risk_free_rate)

    results[ticker] = df_p   # store equity curve and returns
    metrics["Ticker"] = ticker
    metrics_list.append(metrics)

# Combine all metrics into a single DataFrame
metrics_df = (
    pd.concat(metrics_list, axis=0)
      .set_index("Ticker")
)

In [59]:
display(
    metrics_df.style.format(
        {
            "total_return": "{:.2%}",
            "CAGR": "{:.2%}",
            "vol_annual": "{:.2%}",
            "Sharpe": "{:.2f}",
            "max_drawdown": "{:.2%}",
            "hit_ratio": "{:.2%}",
        }
    )
)

,total_return,CAGR,vol_annual,Sharpe,max_drawdown,hit_ratio
Ticker,,,,,,
SPY,653.31%,8.71%,12.72%,0.56,-33.72%,40.21%
QQQ,1326.43%,11.63%,16.17%,0.64,-28.56%,40.31%
NVDA,64650.00%,30.72%,41.48%,0.80,-66.21%,37.03%


In [60]:
final_equity = {
    ticker: results[ticker]["equity_curve"].iloc[-1]
    for ticker in tickers
}

final_equity_df = (
    pd.DataFrame.from_dict(final_equity, orient="index", columns=["final_equity"])
      .rename_axis("Ticker")
)

final_equity_df.style.format({"final_equity": "{:.2f}"})

,final_equity
Ticker,
SPY,7.53
QQQ,14.26
NVDA,647.50


## 2. Equity curves comparison

The equity curve chart shows how the value of a \$1 portfolio evolves over time for each asset when traded with the moving average crossover strategy.

By comparing the curves we can see:

- **Which asset grows faster** under this strategy.  
- **Where the deepest drawdowns occur**, and for which ticker they are most severe.  
- **How volatility impacts the smoothness** of the curve: a choppier equity curve indicates a riskier path, even if the final return is similar.

In general, a smoother equity curve with smaller drawdowns is more attractive from a risk-management perspective, even if the final return is slightly lower.

## 3. Comparison with buy-and-hold and key insights

Although this notebook focuses on the moving average crossover strategy, it is helpful to think about how it would compare with a simple buy-and-hold approach.

- The **moving average crossover tends to reduce drawdowns**, because the strategy exits positions during extended downtrends. However, it does **not necessarily outperform buy-and-hold**, especially for strong momentum assets such as **NVDA**, where staying fully invested for long periods is often rewarded.

- The strategy **trades less during sideways markets**, which decreases exposure when prices chop around the moving averages. This behavior reduces risk, but it can also **limit participation in long-term market trends** if the fast and slow MAs lag behind sharp rallies.

- In many historical samples, **buy-and-hold achieves higher CAGR in growth assets**, whereas the moving average crossover tends to deliver **smoother equity curves and lower volatility**. The choice between these two approaches depends on the investor’s objectives:  
  - If the priority is **maximum long-term growth**, buy-and-hold may be preferable.  
  - If the priority is **reducing large drawdowns and smoothing the ride**, a moving average crossover can be an attractive alternative.

## 4. Conclusion

This project shows how to:

1. Download and clean daily equity data using `yfinance`.  
2. Implement a moving average crossover rule in Python.  
3. Backtest the strategy, compute key performance metrics and visualize equity curves.  

The results highlight the classic trade-off in systematic trading:  
the moving average crossover often **improves risk metrics** (drawdowns, volatility) but does **not always maximize returns** compared with a passive buy-and-hold approach, especially in strong bull markets.

As a next step, we could:

- Optimize the fast/slow windows for each asset.  
- Add transaction costs and slippage.  
- Compare this strategy with other trend-following or mean-reversion rules.

In [63]:
def compute_buy_and_hold(price_series):
    returns = price_series.pct_change().dropna()
    equity = (1 + returns).cumprod()
    return equity

benchmark_results = {}

for ticker in tickers:
    equity_bh = compute_buy_and_hold(prices[ticker])
    benchmark_results[ticker] = equity_bh

comparison_df = metrics_df.copy()

# Total buy-and-hold return per ticker
comparison_df["BH_Total_Return"] = [
    benchmark_results[t].iloc[-1] - 1
    for t in tickers
]

# Buy-and-hold CAGR per ticker
comparison_df["BH_CAGR"] = [
    benchmark_results[t].iloc[-1] ** (252 / len(benchmark_results[t])) - 1
    for t in tickers
]

comparison_df

,total_return,CAGR,vol_annual,Sharpe,max_drawdown,hit_ratio,BH_Total_Return,BH_CAGR
Ticker,,,,,,,,
SPY,6.533082,0.087148,0.127190,0.563684,-0.337173,0.402135,5.322810,0.076707
QQQ,13.264266,0.116251,0.161687,0.637730,-0.285594,0.403120,5.355885,0.076932
NVDA,646.500028,0.307155,0.414772,0.801971,-0.662092,0.370279,1500.459937,0.340612


In [65]:
final_equity = {
    ticker: results[ticker]["equity_curve"].iloc[-1]
    for ticker in tickers
}

final_equity_df = pd.DataFrame.from_dict(final_equity, orient="index", columns=["Final Equity"])
final_equity_df.index.name = "Ticker"
final_equity_df.style.format({"Final Equity": "{:.2f}"})

,Final Equity
Ticker,
SPY,7.53
QQQ,14.26
NVDA,647.50


## Results Analysis

### 1. Overall performance

The table above reports performance metrics for the **moving average crossover strategy** on SPY, QQQ and NVDA between 2000 and 2025. Key observations:

- **Total return and CAGR** show how much each strategy grew a \$1 investment over the full sample.
- **Annualized volatility** captures the risk of the strategy.
- The **Sharpe ratio** measures risk-adjusted performance.
- **Max drawdown** shows the worst peak-to-trough loss over the period.
- The **hit ratio** reports the fraction of profitable days.

In general, we would like to see:
- Positive CAGR
- Sharpe ratio above 1.0 (ideally)
- Reasonable drawdowns compared to the underlying asset.

### 2. Comparison across assets

- **SPY** represents a broad, diversified index; MA crossovers often behave like a trend-following filter, reducing drawdowns at the cost of whipsaws.
- **QQQ** and **NVDA** are more volatile growth assets; the strategy may capture large directional trends but also suffer during sideways periods.

You can comment on:
- Which ticker delivered the highest Sharpe ratio.
- Whether the strategy reduced drawdowns compared to buy-and-hold (a natural extension).
- How sensitive results might be to the choice of window lengths (50/200 vs 20/100, etc.).

### 2.5. Strategy Behavior: MA Crossover vs. Buy-and-Hold

The moving average crossover behaves differently from buy-and-hold, especially during volatile or trending markets. From the results, we observe:

● The MA crossover reduces drawdowns,

because it exits during downtrends when the fast MA crosses below the slow MA.

● However, it does not necessarily outperform buy-and-hold,

especially for strong momentum assets such as NVDA.

● The strategy trades less during sideways markets,

which reduces risk and limits exposure to noise — but also reduces participation in long-term bullish trends.

● Buy-and-hold typically achieves higher CAGR in strong growth assets,

while the MA crossover usually produces smoother equity curves and lower volatility.

This comparison highlights the trade-off between risk reduction and long-term return, which is central to evaluating trend-following strategies.

### 3. Limitations and next steps

This backtest is intentionally simple and has several limitations:
- No transaction costs or slippage.
- Long-only, no shorting.
- Single-asset backtests (no portfolio construction yet).
- Fixed MA parameters across the whole sample.

Possible extensions:
- Add transaction costs and see how performance changes.
- Optimize or grid-search MA windows.
- Build a **multi-asset portfolio** (e.g., equal-weighted SPY/QQQ/NVDA signals).
- Add position sizing and risk management (e.g., volatility targeting).
- Wrap the logic into reusable functions or a small backtesting framework.